In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [2]:
def loadMovieNames():
    movie_names = {}
    with open('/Users/ayusman/migrate/hadooop/hadoop-basics/ml-100k/u.item', encoding = "ISO-8859-1") as f:
        for line in f:
            fields = line.split('|')
            movie_names[int(fields[0])] = fields[1]
    
    return  movie_names

In [3]:
def parseInput(line):
    fields = line.split()
    return (int (fields[1]), float (fields[2]), 1.0)

In [4]:
# setting spark configuration file and spark context
conf = SparkConf().setAppName("worst movies")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/27 14:21:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/27 14:21:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
# getting movie names
movieNames = loadMovieNames()

In [6]:
lines = sc.textFile('/Users/ayusman/migrate/hadooop/hadoop-basics/ml-100k/u.data')

In [7]:
# convert to (movieID, (rating,1.0)) which is and RDD
movieRatings = lines.map(parseInput)

In [106]:
movieRatings.take(2)

[(242, 3.0, 1.0), (302, 3.0, 1.0)]

In [107]:
movieRatings.sortBy(lambda x:x[0])
movieRatings.take(3)


[(242, 3.0, 1.0), (302, 3.0, 1.0), (377, 1.0, 1.0)]

In [94]:
# Reduce to (movieID, (sumofRating, toalRating))
ratingTotalCounts = movieRatings.map(lambda l:(l[0], (l[1], l[2]))).reduceByKey(lambda movie1, movie2: (movie1[0]+movie2[0], (movie1[1]+movie2[1])))

In [108]:
ratingTotalCounts.take(2)

/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[(242, (467.0, 117.0)), (302, (1236.0, 297.0))]

In [111]:
averagePopularRating = ratingTotalCounts.filter(lambda x: x[1][1] > 20)

In [113]:
# Map tp movieID and average rating
averageRating = averagePopularRating.mapValues(lambda totalAndCount: totalAndCount[0]/totalAndCount[1])
type(averageRating)
averageRating.take(2)

[(242, 3.9914529914529915), (302, 4.161616161616162)]

In [114]:
# sort by average rating
sortedAverageRating = averageRating.sortBy(lambda x: x[1])

/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


In [115]:
# these are the worst movies according to their average ratings
sortedAverageRating.take(10)

[(758, 1.7142857142857142),
 (457, 1.7407407407407407),
 (688, 1.8409090909090908),
 (368, 1.903225806451613),
 (1215, 1.9333333333333333),
 (743, 1.9487179487179487),
 (890, 1.9534883720930232),
 (375, 1.9565217391304348),
 (1037, 2.0),
 (564, 2.037037037037037)]